# Entropy estimation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [2]:
lag = 16
stats_count = lag + 4
num_classes = 2
input_data_dim = 20
training_set_ratio = 0.8

## Data-Preprocessing

In [3]:
import math

training_set_ratio = 0.8

with open('random_files/dev-random.bin','rb') as fr:
    fr.seek(0,2)
    training_set_size = math.floor(0.8*fr.tell())
    
    
    fr.seek(0,0)
    data = fr.read(training_set_size)
    

In [4]:
byte_string = "{0:08b}".format(int.from_bytes(data,'big'))

In [5]:
X_D = [float(char) for char in byte_string[:-((training_set_size*8)%(input_data_dim+1))]]

In [6]:
Y = X_D[20::21]

In [7]:
del X_D[20::21]

In [8]:
X_D = torch.tensor(X_D)

In [9]:
X_D = X_D.view(((training_set_size*8)//(input_data_dim+1)), input_data_dim)

In [10]:
X_D

tensor([[1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [11]:
Y = torch.tensor(Y)

In [12]:
Y = Y.view(((training_set_size*8)//(input_data_dim+1)), 1)

In [13]:
Y

tensor([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]])

## Neural Network

In [14]:
class Neural_Network(nn.Module):
    def __init__(self, stats_count, input_data_dim, num_classes):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(stats_count, 20)
        self.fc2 = nn.Linear(20, 1)
        self.fc3 = nn.Linear(input_data_dim+1, 128)
        self.fc4 = nn.Linear(128, 20)
        self.fc5 = nn.Linear(20, num_classes)
    
    
    def forward(self, stats, input_data):
        x = F.relu(self.fc1(stats))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(torch.cat((input_data, x), dim=1)))
        x = F.relu(self.fc4(x))
        x = F.softmax(self.fc5(x), dim=1)        #log_softmax
        return x

model = Neural_Network(stats_count, input_data_dim, num_classes)
print(model)

Neural_Network(
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
  (fc3): Linear(in_features=21, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=2, bias=True)
)


In [15]:
loss_function = nn.CrossEntropyLoss()
total_epochs = 20
learning_rate = 0.001    #update the equation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
X_S = torch.randn(((training_set_size*8)//(input_data_dim+1)), 20)

In [17]:
X_S

tensor([[-4.4725e-01,  1.4553e+00, -2.4143e+00,  ...,  4.6391e-01,
          1.5210e-01,  4.6445e-01],
        [-1.2579e+00,  2.3517e-01,  2.8870e-01,  ...,  1.3206e+00,
          7.3512e-01, -1.2708e-01],
        [-1.1009e+00, -6.1703e-01, -2.1413e+00,  ..., -2.4778e-03,
         -3.8508e-01, -9.6339e-02],
        ...,
        [ 5.9917e-01,  1.0203e+00,  5.1975e-01,  ...,  2.5965e+00,
         -4.1370e-01, -4.7403e-01],
        [-1.4098e+00,  1.3165e+00,  3.4355e-01,  ..., -1.2481e-04,
          1.3352e+00,  6.8080e-02],
        [ 4.5679e-02,  1.0087e+00, -1.5855e+00,  ..., -1.2040e+00,
         -2.7691e-01, -5.6209e-01]])

In [18]:
output = model(X_S,X_D)